<a href="https://colab.research.google.com/github/curio25/recommender-systems/blob/main/recommender_system_content_based_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'the-movies-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3405%2F6663%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240208%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240208T233040Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D04ecb4defc9a4a66473856d7794283c860b7372ad000ac43c42c47995036f6ca2c79fac003bc817ceeb25022dfc473f4e09e4b5b4f963f3af4799025cb95bf604c95b94809856f1e73d007c045ceca1e0f8e33ddbc607f3f15fd314f031a9b7b12017a460a4cc53e2fa4d9e7f4b3b8b705aa6c925af6bc60496f402780636b2b9e1b7c96567c9a2281f56548ba6199ad7c8e253d9552c28b0a18482f1ddd996c97810e0436305582ac6c0bc5f3715882dbf2b726394546913ee4fef318489f39deeb6754143b0898f6ae3c3112dd1e33600f6517acd2b86832c9acfee314e969d852d12a03602604972b50077ce1dabeb5c56639feb74551678a30692aad7103'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 238862293 bytes downloaded
Downloaded and uncompressed: the-movies-dataset
Data source import complete.


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#read data
data_credits = pd.read_csv('/kaggle/input/the-movies-dataset/credits.csv')
data_keywords = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')
data_links = pd.read_csv('/kaggle/input/the-movies-dataset/links.csv')
data_links_small = pd.read_csv('/kaggle/input/the-movies-dataset/links_small.csv')
data_metadata = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
data_ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings.csv')
data_ratings_small = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')

/tmp/ipykernel_33/2091420639.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_metadata = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')


In [ ]:
#peek data
data_credits.head()